Problem 1: Production economy and CO2 taxation

In [2]:
from types import SimpleNamespace
import numpy as np
from scipy.optimize import minimize, root
import pandas as pd
import matplotlib.pyplot as plt


In [12]:
par = SimpleNamespace()
# firms
par.A = 1.0
par.gamma = 0.5
# households
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
# government
par.tau = 0.0
par.T = 0.0
# Question 3
par.kappa = 0.1

#Set w = 1 as numearie
par.w = 1

Define model

In [13]:

def l1_star(par, p1):
    return (p1 * par.A * par.gamma/par.w)**(1 / (1 - par.gamma)) 

def l2_star(par, p2):
    return (p2 * par.A * par.gamma/par.w)**(1 / (1 - par.gamma))

def y1_star(par, l1):
    return par.A * (l1_star(par,p1))**par.gamma

def y2_star(par, l2):
    return par.A * (l2_star(par,p2))**par.gamma

def pi1_star(par, p1):
    return ((1 - par.gamma) / par.gamma) * par.w * (p1 * par.A * par.gamma / par.w)**(1 / (1 - par.gamma))

def pi2_star(par, p2):
    return ((1 - par.gamma) / par.gamma) * par.w * (p2 * par.A * par.gamma / par.w)**(1 / (1 - par.gamma))

def c1(l, par, p1, p2):
    return par.alpha * ((par.w * l + par.T + pi1_star(par, p1) + pi2_star(par, p2)) / p1)

def c2(l, par, p1, p2):
    return (1 - par.alpha) * ((par.w * l + par.T + pi1_star(par, p1) + pi2_star(par, p2)) / (p2 + par.tau))

def l_star(par, p1, p2, initial_guess=0.1):
    def negative_objective(l):
        c1_val = c1(l, par, p1, p2)
        c2_val = c2(l, par, p1, p2)
        if c1_val <= 0 or c2_val <= 0:
            return np.inf  # Penalize invalid solutions
        return -(np.log((c1_val)**par.alpha * (c2_val)**(1 - par.alpha)) - par.nu * l**(1 + par.epsilon) / (1 + par.epsilon))

    result = minimize(negative_objective, initial_guess, method='BFGS')

    # Extract the optimal value of l
    l_star_value = result.x[0]

    return l_star_value


def c1_star(l_star_value, par, p1, p2):
    return c1(l_star_value, par, p1, p2)

def c2_star(l_star_value, par, p1, p2):
    return c2(l_star_value, par, p1, p2)



Define excess demand

In [14]:
def excessdemand_labor(par, p1, p2):
    l1 = l1_star(par, p1)
    l2 = l2_star(par, p2)
    l_opt = l_star(par, p1, p2)
    return l1 + l2 - l_opt


def excessdemand_goodmarket1(par, p1, p2):
    l1 = l1_star(par, p1)
    y1 = y1_star(par, l1)
    l_opt = l_star(par, p1, p2)
    c1_opt = c1_star(l_opt, par, p1, p2)
    return y1 - c1_opt

def excessdemand_goodmarket2(par, p1, p2):
    l2 = l2_star(par, p2)
    y2 = y2_star(par, l2)
    l_opt = l_star(par, p1, p2)
    c2_opt = c2_star(l_opt, par, p1, p2)
    return y2 - c2_opt

Find the excess demand for different p1 and p2 values

In [15]:

# Define ranges for p1 and p2
p1_values = np.linspace(0.1, 2.0, 10)
p2_values = np.linspace(0.1, 2.0, 10)

# Initialize lists to store results
results = []

# Compute excess demands for each combination of p1 and p2
for p1 in p1_values:
    for p2 in p2_values:
        excess_labor = excessdemand_labor(par, p1, p2)
        excess_goodsmarket1 = excessdemand_goodmarket1(par, p1, p2)
        excess_goodsmarket2 = excessdemand_goodmarket2(par, p1, p2)
        results.append((p1, p2, excess_labor, excess_goodsmarket1, excess_goodsmarket2))

# Create a DataFrame for better readability
df_results = pd.DataFrame(results, columns=['p1', 'p2', 'Excess Labor', 'Excess Goods Market 1', 'Excess Goods Market 2'])

# Display the DataFrame
pd.set_option('display.float_format', lambda x: '%.4f' % x)
print(df_results)

# Find the indices where the sign changes
sign_changes = df_results['Excess Goods Market 1'].apply(np.sign).diff().ne(0)

# Filter the indices where the sign changes (excluding the first row, as diff() makes the first value NaN)
sign_change_indices = df_results[sign_changes].index[1:]

print("Indices where sign changes:", sign_change_indices)
print("Values at these indices:")
print(df_results.loc[sign_change_indices])

       p1     p2  Excess Labor  Excess Goods Market 1  Excess Goods Market 2
0  0.1000 0.1000       -0.9933                -2.9600                -6.9734
1  0.1000 0.3111       -0.9645                -3.0036                -2.1347
2  0.1000 0.5222       -0.9063                -3.0930                -1.1432
3  0.1000 0.7333       -0.8194                -3.2300                -0.6770
4  0.1000 0.9444       -0.7047                -3.4171                -0.3844
..    ...    ...           ...                    ...                    ...
95 2.0000 1.1556        0.6328                 0.6948                -0.6549
96 2.0000 1.3667        0.7848                 0.6776                -0.4174
97 2.0000 1.5778        0.9607                 0.6574                -0.2244
98 2.0000 1.7889        1.1599                 0.6340                -0.0604
99 2.0000 2.0000        1.3820                 0.6073                 0.0837

[100 rows x 5 columns]
Indices where sign changes: Index([40, 46, 50], dtyp

**Question 2: Find p1 and p2**

In [16]:
# Define the objective function for finding equilibrium prices
def objectiveprice(p, par):
    p1, p2 = p
    excess_labor = excessdemand_labor(par, p1, p2)
    excess_goodsmarket2 = excessdemand_goodmarket2(par, p1, p2)
    return excess_labor**2 + excess_goodsmarket2**2 #check that labor market clears and good market 2 clears

# List initial guesses in the space to see which would be best
initial_guesses = [
    [0.5, 0.5],
    [1, 1],
    [1.5, 1.5],
    [0.1, 0.1],
    [2.0, 2.0]
]

best_result = None
best_objective_value = np.inf
best_initial_guess = None


# Try initial guesses and find the best result with lowest objective value
for initial_guess in initial_guesses:
    result = minimize(objectiveprice, initial_guess, args=(par,), bounds=[(0.1, 2.0), (0.1, 2.0)])
    if result.fun < best_objective_value:
        best_result = result
        best_objective_value = result.fun
        best_initial_guess = initial_guess

equilibrium_p1, equilibrium_p2 = best_result.x

#Print results for equilibrium prices and the initial guess that gave the best result
print(f"Equilibrium p1: {equilibrium_p1:.4f}, Equilibrium p2: {equilibrium_p2:.4f}")
print(f"Best initial guess: {best_initial_guess}")
print(y2_star(par, l2_star(par, equilibrium_p2)))


Equilibrium p1: 1.3929, Equilibrium p2: 1.1112
Best initial guess: [2.0, 2.0]
1.0


We will now try to plot the optimal behaviour:

**Question 3: What values of tau and T should the government choose to maximize SWF?**

In [17]:

def utility():

    def objec_3(c1, c2, l, p1, p2, par):
        consumption = log(c1(par, l, p1, p2)**par.alpha*c2(par, l, p1,p2)**(1-par.alpha))
        work = par.nu*l**(1-par.epsilon)/(1+par.epsilon)
        u = -(consumption - work)

    # Define the budget constraint
    constraint = p1 * c1 + (p2 + tau) * c2 - (w * ell + T + pi1_star + pi2_star)

    # Initial guess is [0.5,0.5] from before
    x0 = [0.5, 0.5]
    
    

    # Minimize the objective function with the constraint
    result = minimize(objec_3, x0, constraints={'type': 'eq', 'fun': constraint, 'args': (params['p1'], params['p2'], params['w'], params['tau'], params['T'], params['alpha'], params['nu'], params['epsilon'], params['pi1_star'], params['pi2_star'])})

    # Print the results
    print("Optimization Result:", result)
    print(f"c1: {result.x[0]}, c2: {result.x[1]}, ell: {result.x[2]}")




def SWF(U, par, y2_star):

    return U - par.kappa * y2_star

In [18]:
import numpy as np
from scipy.optimize import minimize
from types import SimpleNamespace

# Define parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.T = 0.0
par.kappa = 0.1
par.w = 1  # Set w = 1 as numeraire
par.tau = 0.0  # Default value of tau

# Firm behavior functions
def l_star_firm(par, p):
    return (p * par.A * par.gamma / par.w) ** (1 / (1 - par.gamma))

def y_star_firm(par, p):
    return par.A * (l_star_firm(par, p)) ** par.gamma

def pi_star_firm(par, p):
    return ((1 - par.gamma) / par.gamma) * par.w * (p * par.A * par.gamma / par.w) ** (1 / (1 - par.gamma))

# Consumer utility and social welfare functions
# Consumer utility and social welfare functions
def c1_star3(l_star, par, p1, p2, T):
    return par.alpha * ((par.w * l_star + T + pi_star_firm(par, p1) + pi_star_firm(par, p2)) / p1)

def c2_star3(l_star, par, p1, p2, tau, T):
    return (1 - par.alpha) * ((par.w * l_star + T + pi_star_firm(par, p1) + pi_star_firm(par, p2)) / (p2 + tau))

def utility_all(l_star, par, p1, p2, tau, T):
    c1 = c1_star3(l_star, par, p1, p2, T)
    c2 = c2_star3(l_star, par, p1, p2, tau, T)
    return np.log(c1 ** par.alpha * c2 ** (1 - par.alpha)) - par.nu * l_star ** (1 + par.epsilon) / (1 + par.epsilon)

def optimize_l_star(par, p1, p2, tau, T):
    def neg_utility_all(l):
        return -utility_all(l, par, p1, p2, tau, T)
    
    initial_guess = 0.1
    result = minimize(neg_utility_all, initial_guess, bounds=[(0, None)], method='SLSQP')
    return result.x[0] if result.success else None

def social_welfare(vars, par, p1, p2, optimize_tau=False):
    if optimize_tau:
        tau = vars[0]
    else:
        tau = par.tau  # Use fixed tau from par

    # Initial guess for T
    T_guess = 0.1
    l_star = optimize_l_star(par, p1, p2, tau, T_guess)
    if l_star is None:
        return np.inf  # Return a large number if optimization fails

    # Iteratively update T based on c2_star
    for _ in range(10):
        c2 = c2_star3(l_star, par, p1, p2, tau, T_guess)
        T_guess = tau * c2
        l_star = optimize_l_star(par, p1, p2, tau, T_guess)
        if l_star is None:
            return np.inf

    y2_star = y_star_firm(par, p2)
    U = utility_all(l_star, par, p1, p2, tau, T_guess)
    SWF = U - par.kappa * y2_star
    
    print("Value of y2_star:", y2_star)
    print("Value of U:", U)
    print("Value of SWF:", SWF)
    print("Value of c2:", c2)
    return -SWF  # We minimize the negative SWF to maximize SWF


# Function to optimize tau and T implicitly through the government budget constraint
def optimize_tau(par, p1, p2):
    initial_guess = [0.1]
    bounds = [(0, None)]
    result = minimize(social_welfare, initial_guess, args=(par, p1, p2, True), bounds=bounds, method='SLSQP')
    if result.success:
        tau_opt = result.x[0]
        l_star = optimize_l_star(par, p1, p2, tau_opt, 0.1)  # Initial guess for T
        if l_star is not None:
            c2_opt = c2_star3(l_star, par, p1, p2, tau_opt, 0.1)
            T_opt = tau_opt * c2_opt
            return tau_opt, T_opt
    print("Optimization failed:", result.message)
    return None

# Example usage (maybe use equilibrium prices)
p1 = equilibrium_p1
p2 = equilibrium_p2

# Optimize tau (T is determined by the budget constraint)
optimal_tau_T = optimize_tau(par, p1, p2)
if optimal_tau_T is not None:
    optimal_tau, optimal_T = optimal_tau_T
    print(f"Optimal tau: {optimal_tau:.4f}, Optimal T: {optimal_T:.4f}")






Value of y2_star: 0.5555873151894014
Value of U: -0.4897573121820785
Value of SWF: -0.5453160437010187
Value of c2: 0.9624957130182825
Value of y2_star: 0.5555873151894014
Value of U: -0.4897573124129999
Value of SWF: -0.54531604393194
Value of c2: 0.9624957090661598
Value of y2_star: 0.5555873151894014
Value of U: -0.48944634796862785
Value of SWF: -0.545005079487568
Value of c2: 0.9681320959914042
Value of y2_star: 0.5555873151894014
Value of U: -0.4894463481929462
Value of SWF: -0.5450050797118864
Value of c2: 0.9681320915647438
Value of y2_star: 0.5555873151894014
Value of U: -0.488647415740036
Value of SWF: -0.5442061472589761
Value of c2: 0.9967047219967057
Value of y2_star: 0.5555873151894014
Value of U: -0.48864741575025106
Value of SWF: -0.5442061472691913
Value of c2: 0.9967047196399054
Value of y2_star: 0.5555873151894014
Value of U: -0.4886415913134728
Value of SWF: -0.5442003228324129
Value of c2: 0.9981202816803991
Value of y2_star: 0.5555873151894014
Value of U: -0.48864

In [7]:

# Define parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.T = 0.0
par.kappa = 0.1
par.w = 1  # Set w = 1 as numeraire
par.tau = 0.0  # Default value of tau
par.l = 10.0  # Example value for labor supply

# Firm behavior functions
def l_star_firm(par, p):
    return (p * par.A * par.gamma / par.w) ** (1 / (1 - par.gamma))

def y_star_firm(par, p):
    return par.A * (l_star_firm(par, p)) ** par.gamma

def pi_star_firm(par, p):
    return ((1 - par.gamma) / par.gamma) * par.w * (p * par.A * par.gamma / par.w) ** (1 / (1 - par.gamma))

# Consumer utility functions
def c13(par, p1, p2, T, l):
    return par.alpha * ((par.w * l + T + pi_star_firm(par, p1) + pi_star_firm(par, p2)) / p1)

def c23(par, p1, p2, tau, T, l):
    return (1 - par.alpha) * ((par.w * l + T + pi_star_firm(par, p1) + pi_star_firm(par, p2)) / (p2 + tau))

def l_star_consumer(par, p1, p2, tau, T):
    # Objective function to minimize
    def lstar(l):
        c1 = c13(par, p1, p2, T, l)
        c2 = c23(par, p1, p2, tau, T, l)
        return -(np.log(c1**par.alpha * c2**(1-par.alpha)) - par.nu*l**(1+par.epsilon)/(1+par.epsilon))
    
    # Optimize
    initial_guess = [0.1]
    bounds = [(0, None)]
    result = minimize(lstar, initial_guess, bounds=bounds, method='SLSQP')
    return result.x[0]

def utility_all(par, p1, p2, tau, T):
    l_star = l_star_consumer(par, p1, p2, tau, T)
    c1 = c13(par, p1, p2, T, l_star)
    c2 = c23(par, p1, p2, tau, T, l_star)
    return np.log(c1 ** par.alpha * c2 ** (1 - par.alpha)) - par.nu * l_star ** (1 + par.epsilon) / (1 + par.epsilon)

def social_welfare(par, p1, p2, tau, T):
    utility = utility_all(par, p1, p2, tau, T)
    swf = utility + par.kappa * y_star_firm(par, p2)
    return -swf

def optimize_tau(par, p1, p2):
    initial_guess = [0.1]
    bounds = [(0, None)]
    result = minimize(lambda tau: social_welfare(par, p1, p2, tau, par.T), initial_guess, bounds=bounds, method='SLSQP')
    return result.x[0]

p1 = 2
p2 = 1

result = optimize_tau(par, p1, p2)
print(result)

2.7755575615628914e-17


In [ ]:
Calculate T

In [11]:
tau_optimal = optimize_tau(par, p1, p2)
T_optimal = tau_optimal * c23(par, p1, p2, tau_optimal, 0, l_star_consumer(par, p1, p2, tau_optimal, 0))

print("Implied T:", T_optimal)

Implied T: 2.6385982016519682e-17
